In [1]:
import logging
import numpy as np
import yaafelib
import librosa
from aubio import onset
from appconfig import setup_logging
from scipy.io import wavfile
import matplotlib.pyplot as plt

%matplotlib inline  

featurespecs = \
    ['LSF: LSF blockSize={} stepSize={} LSFNbCoeffs=10 LSFDisplacement=1',
     'MelSpectrum: MelSpectrum blockSize={} stepSize={}',
     'SpectralCrestFactorPerBand: SpectralCrestFactorPerBand blockSize={} stepSize={}',
     'SpectralDecrease: SpectralDecrease blockSize={} stepSize={}',
     'SpectralFlatnessPerBand: SpectralFlatnessPerBand blockSize={} stepSize={}',
     'SpectralFlux: SpectralFlux blockSize={} stepSize={}',
     'SpectralSlope: SpectralSlope blockSize={} stepSize={}',
     'SpectralRolloff: SpectralRolloff blockSize={} stepSize={}']

In [2]:
def get_onsets(signal, sr, nfft, hop, onset_detector_type, onset_threshold=None):
    onsets = []

    onset_detector = onset('default', nfft, hop, sr)
    if onset_threshold:
        onset_detector.set_threshold(onset_threshold)

    signal_windowed = np.array_split(signal, np.arange(hop, len(signal), hop))

    for frame in signal_windowed[:-1]:
        if onset_detector(frame):
            onsets.append(onset_detector.get_last_s())
    return onsets[1:]

In [3]:
path = '/home/tracek/Data/gender/test/ablackball-20121113-vvk/wav/a0333.wav'
path2 = '/home/tracek/Data/gender/Voxforge/ralfherzog-20070923-en8/flac/en8-013.flac'

sr = 16000
block_size = 1024
feature_plan = yaafelib.FeaturePlan(sample_rate=sr, normalize=True)
for featurespec in featurespecs:
    feature = featurespec.format(block_size, block_size // 2)
    assert feature_plan.addFeature(feature), 'Failed to load %s feature' % feature
    logging.info('Feature %s loaded', feature)
    
engine = yaafelib.Engine()
engine.load(feature_plan.getDataFlow())
afp = yaafelib.AudioFileProcessor()

In [4]:
y, sr = librosa.load(path, sr=sr)
y = librosa.util.normalize(y)
y = y.astype('float32')

In [5]:
get_onsets(y, sr, nfft=512, hop=256, onset_detector_type='hfc', onset_threshold=0.3)

[0.9438750147819519,
 1.0125625133514404,
 1.1795624494552612,
 1.309999942779541,
 1.5658750534057617,
 1.715999960899353]

In [6]:
bw = get_onsets(y[::-1], sr, nfft=512, hop=256, onset_detector_type='hfc', onset_threshold=0.3)
(len(y) / sr  - np.array(bw)[::-1])

array([ 1.06925011,  1.21224999,  1.40681243,  1.77150011])

In [ ]:
len(y) / sr

In [ ]:
signal_windowed = np.array_split(y, np.arange(256, len(y), 256))

In [ ]:
onset_samples_fw = librosa.onset.onset_detect(y=y, sr=sr, hop_length=256, units='samples')
onset_samples_bw = librosa.onset.onset_detect(y=y[::-1], sr=sr, hop_length=256, units='samples')
onset_samples_bw = (len(y) - onset_samples_bw)[::-1]
print('Forward: ', librosa.core.samples_to_time(onset_samples_fw, sr))
print('Backward: ', librosa.core.samples_to_time(onset_samples_bw, sr))

In [ ]:
onset_frames_fw = librosa.onset.onset_detect(y=y, sr=sr, hop_length=256, units='samples')

In [7]:
max(0, 3)

3